# Fatos e mitos sobre referências e objetos em Python

Este notebook é a segunda parte de uma adaptação do post ["Facts and myths about Python names and values"](https://nedbatchelder.com/text/names.html), de Ned Batchelder.

## A diversidade do Python

### Fato: referências podem ser mais que apenas nomes.

Todos os exemplos usados até aqui usaram nomes como referências a objetos, mas outras coisas também podem ser referências.

Python tem um leque de estruturas de dados compostas, cada uma armazenando referências a objetos: elementos de uma lista, chaves e valores de um dicionário, atributos de objetos, e por aí vai.

Cada exemplo acima pode ser usado do lado esquerdo de uma operação de associação e todos os detalhes discutos para associação também se aplicam aqui.

Em resumo: **qualquer coisa que possa aparecer do lado esquerdo de uma associação é uma referência** e você pode substituir o **nome** pelo termo **referência** em todos os lugares onde ele aparecia.

No nosso diagrama de listas, eu mostrei números como elementos, mas na verdade cada elemento é uma referência a número. 

A ilustração mais apropriada é esta:

<img src="imgs/ned10.png">

Por simplicidade, vamos continuar usando o estilo de ilustração anterior.

Se você tiver elementos de uma lista se referindo a outros objetos mutáveis, como sub-listas, é importante se lembrar que os elementos da lista são apenas referências a objetos.

Veja mais alguns exemplos de associações. Cada um desses elementos que aparecem do lado esquerdo das operações de associação é uma referência:

In [1]:
my_dict = {}
my_dict["a"] = 24
print(my_dict)

{'a': 24}


In [2]:
my_list = [1, 2, 3]
my_list[0] = 25
print(my_list)

[25, 2, 3]


Muitas estruturas de Python armazenam objetos, cada um deles através de uma referência.

Todas as regras explicadas aqui sobre nomes se aplicam exatamente da mesma forma a qualquer uma dessas referências.

O coletor de lixo, por exemplo, não conta apenas nomes -- conta qualquer tipo de referência para decidir quando um objeto pode ser liberado.

Note que ```i = x``` faz uma associação do nome **i** com o objeto referenciado por **x**. Já ```i[0] = x``` faz uma associação da referência presente na primeira posição da lista referenciada por **i** com o objeto referenciado por **x**.

É importante entender corretamente o que está sendo associado com o quê. Só porque um nome aparece em alguma parte do lado esquerdo de uma associação, isso não significa que o nome esteja sendo reassociado.

### Fato: muitas coisas são associações

Assim como muitas coisas podem servir como referências, muitas operações em Python são associações. 

Cada uma das linhas a seguir é uma associação com o nome **X**:

In [3]:
for X in "texto":
    print(id(X))

4384446088
4384801544
4384150672
4384446088
4385238184


In [4]:
[id(X) for X in "texto"]

[4384446088, 4384801544, 4384150672, 4384446088, 4385238184]

In [5]:
{id(X) for X in "texto"}

{4384150672, 4384446088, 4384801544, 4385238184}

In [6]:
class X():
    pass
print(id(X))

140214689169640


In [7]:
def X():
    pass
print(id(X))

4411468656


In [8]:
def fn(X): 
    print(id(X))
fn(12)

4383079328


In [9]:
with open("imgs/ned1.png") as X:
    print(id(X))

4410248432


In [10]:
import random
print(id(random))

4389861240


In [11]:
import random as X
print(id(X))

4389861240


In [12]:
from random import randint
print(id(randint))

4389391944


In [13]:
from random import randint as X
print(id(X))

4389391944


In [14]:
try:
    some_dict["a"] = 1
except NameError as X:
    print(id(X))

4411991496


Eu não quero dizer que essas instruções atuam como associações -- eu quero dizer que elas são associações.

Todas elas fazem com o que o nome **X** (ou ```random``` e ```randint```, nos exemplos 23 e 25) se refira a um objeto e tudo o que eu tenho dito sobre associações se aplicam a elas igualmente.

### Fato: Python passa argumentos de função através de associações.

Vamos analisar o caso mais interessante dentre essas associações: chamar um procedimento.

Quando eu defino um procedimento, eu nomeio seus parâmetros:

In [15]:
def my_proc(x, y):
    return x+y

Aqui, **x** e **y** são parâmetros da função referenciada pelo nome ```my_proc``` -- lembre-se que em Python tudo é objeto ou referência, logo um procedimento é um objeto cujo nome é uma referência.

Ao invocar ```my_proc```, eu forneço objetos existentes para serem usados como parâmetros do procedimento. Estes objetos são associados aos nomes definidos para os parâmetros da mesma forma que uma instrução de associação simples faria:

In [16]:
def my_proc(x, y):
    return x+y

print(my_proc(8, 9))

17


Quando ```my_proc``` é invocada, o nome **x** é associado ao objeto **8**, e o nome **y** é associado ao objeto **9**. Esta associação funciona exatamente como as instruções de associação que discutimos. Os nome **x** e **y** são locais ao procedimento, então quando o procedimento retorna, estes nomes desaparecem. No entanto, se os objetos aos quais eles estavam associados estiverem sendo referidos por outros nomes, estes objetos não desaparecerão.

Assim como em qualquer outra associação, objetos mutáveis podem ser passados como parâmetros de procedimentos, e mudanças feitas nesses objetos serão visíveis para todos os seus nomes:

In [1]:
def augment_twice(a_list, val):
    """Acrescenta duas referências a `val` no final da lista `a_list`."""
    a_list.append(val)
    a_list.append(val)

nums = [1, 2, 3]
augment_twice(nums, 4)
print(nums)

[1, 2, 3, 4, 4]


Talvez o resultado da execução do código acima seja surpreendente para você, então vamos analisá-lo passo a passo. Quando invocamos o procedimento ```augment_twice```, os nomes e objetos estão assim:

<img src="imgs/ned11.png">

As referências locais em um procedimento são desenhados em uma moldura. Invocar o procedimento associou os objetos aos nomes dos parâmetros, assim como qualquer instrução de associação faria.

Lembre-se que associações nunca cria novos objetos nem copia dado algum, então aqui a referência local ```a_list``` está associada ao mesmo objeto passado na invocação -- isto é, o objeto ao qual ```nums``` também está associado.

Neste ponto, chamamos a_list.append duas vezes, que altera a lista:

<img src="imgs/ned12.png">

Quando o procedimento se encerra, as referências locais são destruídas. Objetos que não sejam mais referenciados são coletados, mas os outros permanecem:

<img src="imgs/ned13.png">

Nós passamos a lista para o procedimento, que a modificou. Nenhum objeto foi copiado. 

Ainda que este comportamento possa ser surpreendente, ela é essencial Sem ela, não seria possível criar métodos que alterassem objetos.

Aqui vai uma outra forma de criar um procedimento, mas que não funciona. Vamos ver porquê:

In [2]:
def augment_twice_bad(a_list, val):
    """Acrescenta duas referências a `val` no final da lista `a_list`."""
    a_list = a_list + [val, val]

nums = [1, 2, 3]
augment_twice_bad(nums, 4)
print(nums)

[1, 2, 3]


No momento em que invocamos ```augment_twice_bad```, tudo parece igual ao exemplo anterior:

<img src="imgs/ned14.png">

A próxima instrução é uma associação.

A expressão no lado direito cria uma nova lista, que então é associada à referência ```a_list```.

**Lembre-se que os operadores infixos não tentam fazer alterações in-place! Este papel é dos operadores de associação composta!**

<img src="imgs/ned15.png">

Quando o procedimento acaba, suas referências locais são destruídas e quaisquer objetos que não sejam mais referenciados são coletados, nos levando de volta ao estado anterior à execução do procedimento:

<img src="imgs/ned16.png">

É realmente importante ter em mente a diferença entre alterar um objeto in-place e reassociar uma referência.

O procedimento ```augment_twice``` funcionou porque alterou o objeto passado, então a alteração estava disponível após o procedimento retornar.

Já o procedimento ```augment_twice_bad``` usou uma associação para reassociar uma referência local, então estas mudanças não são visíveis fora do procedimento.

Outra opção para o nosso procedimento é criar um novo objeto e retorná-lo:

In [4]:
def augment_twice_good(a_list, val):
    a_list = a_list + [val, val]
    return a_list

nums = [1, 2, 3]
nums = augment_twice_good(nums, 4)
print(nums)

[1, 2, 3, 4, 4]


Aqui nós criamos um objeto inteiramente novo dentro de ```augment_twice_good``` e o retornamos de dentro do procedimento. Quem chamou o procedimento usa uma associação para se referir a esse objeto, então conseguimos o efeito que queremos.

Este último procedimento é talvez o melhor, já que cria a menor quantidade de surpresas, uma vez que não tenta fazer alterações in-place, criando diretamente novos objetos.

Não há resposta certa entre escolher alterar ou reassociar: o que você deve usar depende do efeito que você precisa.

O mais importante é entender como cada situação acontece, saber as ferramentas à sua disposição e então escolher a que funcione melhor para o seu problema específico.

## Tipagem dinâmica

Alguns detalhes sobre os objetos e referências Python:

### Fato: Qualquer nome pode se referir a qualquer objeto em qualquer momento.

Python tem tipagem dinâmica, o que significa que referências em si não tem tipos. 

Qualquer nome pode se referir a qualquer objeto em qualquer momento. 

Um nome pode se referir a um inteiro, depois a uma string, em seguida a um procedimento e, por fim, a um módulo.

Obviamente, este seria um código bastante confuso, então **não faça isso**, mas o interpretador Python não se confundirá!

### Fato: Referências não têm tipo, objetos não tem escopo.

Assim como referências não têm tipo, objetos não têm escopo.

Quando dizemos que um procedimento tem uma variável local, queremos dizer que aquela referência tem seu escopo restrito ao procedimento: você não consegue usá-la fora do procedimento e, quando o procedimento retornar, aquela referência será destruída.

Mas como já vimos, se o objeto referido internamente no procedimento tiver referências externas ao procedimento, ele continuará existindo mesmo quando o procedimento retornar. 

**É uma referência local, não um objeto local.**

In [3]:
def proc(a):
    a.append(2)
    print(id(a))
    
b = [1,3]
proc(b)
print(id(b))
print(b)

4448867976
4448867976
[1, 3, 2]


### Fato: Objetos não podem ser destruídos, apenas referências.

A gerência de memória do Python é central para o comportamento do interpretador -- **não só você não têm que destruir objetos, você não tem como destruir objetos**.

Talvez você tenha lido sobre a instrução ```del```:

In [6]:
nums = [1, 2, 3]
del nums

Isto não destrói o objeto referido por **nums** -- isto destrói a referência **nums**.

Esta referência é removida do escopo atual e então o coletor de lixo do Python entra em ação: se o objeto referido por **nums** só estivesse sendo referido por **nums**, este objeto será coletado. Caso contrário, o objeto continua existindo.

### Mito: Python não têm variáveis.

Algumas pessoas gostam de dizer que "*Python não tem variáveis --  apenas nomes*". 

people like to say, “Python has no variables, it has names.” This slogan is misleading. The truth is that Python has variables, they just work differently than variables in C.

Names are Python’s variables: they refer to values, and those values can change (vary) over the course of your program. Just because another language (albeit an important one) behaves differently is no reason to describe Python as not having variables.


